In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

flights = pd.read_csv("C:/Users/kenne/Downloads/flights_15_cities_full_routes.csv")

# Parse dates to get into consistent format
flights['date_dt'] = pd.to_datetime(flights['date']).dt.date
cities = sorted(flights['destination'].unique())
dates = sorted(flights['date_dt'].unique())

# Parameters
L = 3  # Maximum days staying in a city (including concert day)
START_CITY = "LAX"  # Fixed starting city

m = gp.Model("WorldTourScheduling")

# Decision Variables
X = {}  # Flights
for i, row in flights.iterrows():
    f_id = row['flight_id']
    X[f_id] = m.addVar(vtype=GRB.BINARY, name=f"X_{f_id}")

Y = {}  # Concerts
for c in cities:
    for d in dates:
        Y[c, d] = m.addVar(vtype=GRB.BINARY, name=f"Y_{c}_{d}")

# Objective: minimize total flight cost
m.setObjective(sum(X[row['flight_id']] * row['price_usd'] for i, row in flights.iterrows()), GRB.MINIMIZE)

# Constraints

# 1. Each city must have exactly one concert
for c in cities:
    if c == START_CITY:
        continue
    m.addConstr(sum(Y[c, d] for d in dates) == 1, name=f"OneConcert_{c}")

# No concerts in START_CITY (First date added manually since we don’t need to schedule flight)
for d in dates:
    m.addConstr(Y[START_CITY, d] == 0, name="NoConcertAtStartCity")

# 2. Each city must have exactly one incoming flight
for c in cities:
    if c == START_CITY:
        continue # START_CITY incoming flight handled in later return-flight constraint
    relevant_flights = flights[flights['destination'] == c]['flight_id'].tolist()
    m.addConstr(sum(X[f] for f in relevant_flights) == 1, name=f"OneFlight_{c}")

# 3. Flight-concert linkage (arrival 1 day before to L-1 days after)
flight_allowed_dates = {}
for i, row in flights.iterrows():
    f_id = row['flight_id']
    c = row['destination']
    f_date = row['date_dt']

    if c == START_CITY:
        continue  # returning to start city doesn’t need a concert

    min_concert_date = f_date + pd.Timedelta(days=1)
    max_concert_date = f_date + pd.Timedelta(days=L-1)

    # getting all allowed dates for each concert
    for d in dates:
        if min_concert_date <= d <= max_concert_date:
            if f_id not in flight_allowed_dates:
                flight_allowed_dates[f_id] = []
            flight_allowed_dates[f_id].append(d)

for i, row in flights.iterrows():
    f_id = row['flight_id']
    c = row['destination']
    if c == START_CITY:
        continue
    allowed = flight_allowed_dates.get(f_id, [])
    if not allowed:
        m.addConstr(X[f_id] == 0, name=f"NoFeasibleDates_{f_id}")
    else:
        m.addConstr(X[f_id] <= sum(Y[c, d] for d in allowed), name=f"ArrivalConcertWindow_{f_id}")

# 4. No more than one concert per day globally
for d in dates:
    m.addConstr(sum(Y[c, d] for c in cities) <= 1, name=f"OneConcertPerDay_{d}")

# 5. At most one flight per day
for d in dates:
    m.addConstr(sum(X[row['flight_id']] for i, row in flights.iterrows() if row['date_dt'] == d) <= 1, name=f"OneFlightPerDay_{d}")

# 6. Origin constraint: flights must start from previous city
for i, row in flights.iterrows():
    f_id = row['flight_id']
    origin = row['origin']
    dest = row['destination']
    f_date = row['date_dt']

    # First flight must start from START_CITY
    if f_date == min(dates):
        if origin != START_CITY:
            m.addConstr(X[f_id] == 0, name=f"FirstFlightFromSTART_{f_id}")
        continue

    # Returning to START_CITY doesn’t need origin check here
    if dest == START_CITY:
        continue

    prev_day = f_date - pd.Timedelta(days=1)
    m.addConstr(X[f_id] <= sum(Y[origin, d] for d in [prev_day] if d in dates), name=f"FlightOriginCheck_{f_id}")

# 7. Ensure exactly one return flight to START_CITY
return_flights = flights[flights['destination'] == START_CITY]['flight_id'].tolist()
m.addConstr(sum(X[f] for f in return_flights) == 1, name="ReturnToStartCity")

# enforce return flight originates from previous concert city
for f_id in return_flights:
    origin = flights.loc[flights['flight_id'] == f_id, 'origin'].values[0]
    f_date = flights.loc[flights['flight_id'] == f_id, 'date_dt'].values[0]
    prev_day = f_date - pd.Timedelta(days=1)
    if prev_day in dates:
        m.addConstr(X[f_id] <= sum(Y[origin, d] for d in [prev_day]), name=f"ReturnOriginCheck_{f_id}")

# Solve model
m.optimize()

# Output results
if m.status == GRB.OPTIMAL:
    print("\nTour Schedule:\n")
    print("Concert in LAX on 2025-04-30") #hard-coded start-city concert
    concert_list = []
    for c in cities:
        for d in dates:
            if Y[c, d].X > 0.5:
                concert_list.append((d, c))
    concert_list.sort()
    for d, c in concert_list:
        print(f"Concert in {c} on {d}")

    print("\nSelected Flights:\n")
    # Collect selected flights
    selected_flights = []
    for i, row in flights.iterrows():
        f_id = row["flight_id"]
        if X[f_id].X > 0.5:
            selected_flights.append((row['date_dt'], f_id, row['origin'], row['destination'], row['price_usd']))
    
    # Sort flights by date
    selected_flights.sort()
    
    # Print sorted flights
    for f_date, f_id, origin, dest, price in selected_flights:
        print(f"Flight {f_id} from {origin} to {dest} on {f_date} (Cost: {price})")
else:
    print("No feasible solution found")

Set parameter Username
Set parameter LicenseID to value 2700353
Academic license - for non-commercial use only - expires 2026-08-27
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: Intel(R) Core(TM) Ultra 7 155U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 25373 rows, 13950 columns and 89584 nonzeros
Model fingerprint: 0xb1a7edc0
Variable types: 0 continuous, 13950 integer (13950 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e+01, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 13799 rows and 1504 columns
Presolve time: 1.47s
Presolved: 11574 rows, 12446 columns, 69202 nonzeros
Variable types: 0 continuous, 12446 integer (12446 binary)

Root relaxation: objective 3.020038e+03, 3624 iterations, 1.03 seconds (0.85 work units)

    Nodes    |    Current Node    |     Obje